### Import Packages

In [ ]:
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

from scipy.stats import pointbiserialr, chi2_contingency, uniform, randint
from itertools import combinations

### Gather Data

In [ ]:
# Training and Validation Data
train_file = ""
df = pd.read_excel(train_file)
df.set_index('CID', inplace= True)

X, y = df.iloc[:, :-1], df.iloc[:,-1]
df_train_x, df_val_x, y_train, y_val = train_test_split(X, y, test_size= 0.1, random_state= 42, stratify= y)

print("\n\033[1mTraining Data information: \033[0m\n")
df_train_x.info()
print("\n\033[1mTraining Label information: \033[0m\n")
y_train.info()
label_count_tr = y_train.value_counts()
print("\n\033[1mIn the Training Dataset:\033[0m\n")
print(f"Percentage of positively labeled data: {(label_count_tr[1] / (label_count_tr[0] + label_count_tr[1])) * 100}%")

print("\n\033[1mValidation Data information: \033[0m\n")
df_val_x.info()
print("\n\033[1mValidation Label information: \033[0m\n")
y_val.info()
label_count_ts = y_val.value_counts()
print("\n\033[1mIn the Validation Dataset: \033[0m\n")
print(f"Percentage of positively labeled data: {(label_count_ts[1] / (label_count_ts[0] + label_count_ts[1])) * 100}%")

# Test Data
test_file = ""
df_test = pd.read_excel(test_file)
df_test.set_index('CID', inplace= True)

### Pipeline Custom Classes

In [ ]:
class FeatureDropper(BaseEstimator, TransformerMixin):
    
    def __init__(self, col_to_drop):
        self.columns_to_drop = col_to_drop
    
    def fit(self, X, y= None):
        return self
    
    def transform(self, X):
        return X.drop(columns= self.columns_to_drop)

class OneHotEncoding(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns, binary= False):
        self.columns = columns
        self.binary = binary
        self.encoders = {}
        self.new_column_names = {}
        
    def fit(self, X, y= None):
        X_transformed = X.copy()
        for column in self.columns:
            if self.binary:                
                category_names = sorted(list(X_transformed[column].drop_duplicates()))
                self.binary_check(column, category_names)
                self.encoders[column] = {col: id for id, col in enumerate(category_names)}
            else:
                category_names = sorted([x for x in X_transformed[column].drop_duplicates() 
                                         if not (isinstance(x, float) and math.isnan(x))])
                self.new_column_names[column] = [f"{column}_{c}" for c in category_names]
                encoder = OneHotEncoder(dtype= np.int64, drop= 'if_binary')
                encoder.fit(X_transformed[[column]])
                self.encoders[column] = encoder
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for column in self.columns:
            if self.binary:
                X_transformed[column] = X_transformed[column].map(self.encoders[column])
            else:
                ohe_matrix = self.encoders[column].transform(X_transformed[[column]]).toarray()
                if self.has_nan(X_transformed[column]):
                    ohe_matrix = ohe_matrix[:, :-1]
                for i, new_col in enumerate(self.new_column_names[column]):
                    X_transformed.insert(X_transformed.columns.get_loc(column) + i, new_col, ohe_matrix[:, i])
                X_transformed.drop(columns= [column], inplace= True)
        return X_transformed
    
    def binary_check(self, column, categories):
        if self.has_nan(categories):
            raise ValueError(f"Can't perform binary encoding to column {column} because there are NaN values.")
        if len(categories) > 2 or len(categories) == 0:
            raise ValueError(f"Can't perform binary encoding to column {column} because the number of categories to binary encode is wrong.")

    def has_nan(self, value_list):
        return any([math.isnan(x) for x in value_list if isinstance(x, float)])

class TargetMeanEncoding(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns, y, m= 10):
        self.columns = columns
        self.y_target = y
        self.m = m
        self.encoding_dict = {}
        
    def fit(self, X, y= None):
        global_mean = self.y_target.mean()
        for column in self.columns:
            smoothed_mean = self.smooth_mean(X[column], global_mean)
            self.encoding_dict[column] = dict(zip(X[column].dropna().drop_duplicates(), smoothed_mean.dropna().drop_duplicates()))        
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        global_mean = self.y_target.mean()
        
        for column in self.columns:
            if column in self.encoding_dict:
                X_transformed[column] = X_transformed[column].map(self.encoding_dict[column]).fillna(global_mean)
            else:
                X_transformed[column] = X_transformed[column].fillna(global_mean)
                
        return X_transformed
    
    def smooth_mean(self, values, mean):
        encoded_mean = self.y_target.groupby(values).mean()
        counts = values.map(values.value_counts())
        return (values.map(encoded_mean) * counts + mean * self.m) / (counts + self.m)
    
class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns, scaler_cls, **scaler_params):
        self.columns = columns
        self.scaler_cls = scaler_cls
        self.scaler_params = scaler_params
        self.scalers = {}
        
    def fit(self, X, y= None):
        for col in self.columns:
            scaler = self.scaler_cls(**self.scaler_params)
            scaler.fit(X[[col]])
            self.scalers[col] = scaler
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            X_copy[col] = self.scalers[col].transform(X_copy[[col]])
        return X_copy

class PolynomialFeature(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns, degree= 2):
        self.columns = columns
        self.degree = degree

    def fit(self, X, y= None):
        return self
            
    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            for power in range(2, self.degree + 1):
                new_col_name = f"{col}_pow{power}"
                X_copy[new_col_name] = X_copy[col] ** power
        return X_copy

class LogTransformFeature(BaseEstimator, TransformerMixin):
            
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y= None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            new_col_name = f"{col}_log"
            X_copy[new_col_name] = np.log1p(X_copy[col])
        return X_copy
            
class InteractionFeature(BaseEstimator, TransformerMixin):
            
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y= None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        for (col1, col2) in combinations(self.columns, 2):
            new_col_name = f"{col1}_x_{col2}"
            X_copy[new_col_name] = X_copy[col1] * X_copy[col2]
        return X_copy
        
class BinningFeature(BaseEstimator, TransformerMixin):
            
    def __init__(self, columns, bins= 5):
        self.columns = columns
        self.bins = bins

    def fit(self, X, y= None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            new_col_name = f"{col}_binned"
            X_copy[new_col_name], _ = pd.cut(X_copy[col], bins= self.bins, retbins= True, labels= False)
        return X_copy

### Investigating Features
##### Examining Correlations of Features

In [ ]:
df.head(5)

In [ ]:
unique_categories = {}
for feature in df.columns[12:-1]:
    unique_categories[feature] = X[feature].nunique()
    
for feature, count in unique_categories.items():
    print(f"Feature {feature} has {count} unique values.")

In [ ]:
corr_pipeline = Pipeline([
    ("Binary Encoder", OneHotEncoding(["A"], binary= True)),
    ("One-Hot Encoder", OneHotEncoding(["B", "C", "D"])),
    ("Target-Mean Encoder", TargetMeanEncoding(["E"], y_train))
])
df_corr = corr_pipeline.fit_transform(df)

correlation_matrix = df_corr.corr()
plt.figure(figsize= (30, 24))
sns.heatmap(correlation_matrix, annot= True, cmap= 'coolwarm', fmt= '.2f')
plt.title('Correlation Matrix')
plt.show()

correlation_with_target = df_corr.corr()['Target'].sort_values(ascending= False)
print(correlation_with_target)

In [ ]:
positive_label = df_corr[df_corr['Target'] == 1]
negative_label = df_corr[df_corr['Target'] == 0]

for column in df_corr.columns[:-1]:
    plt.figure(figsize= (5, 3))
    sns.histplot(positive_label[column], bins= 15, color= 'b', kde= True, label= '1')
    sns.histplot(negative_label[column], bins= 15, color= 'r', kde= True, label= '0')
    plt.title(f"Histogram of {column}:")
    plt.legend()
    plt.show()

##### Detecting Outliers

In [ ]:
outlier_pipeline = Pipeline([
    ("Feature Dropper", FeatureDropper(["A", "B", "C", "D", "E", "F",
                                        #"G", "H", "I",
                                        "J", "K", "L", "M", "N", "O", "U", "P"
                                       ])),
])

df_out = outlier_pipeline.fit_transform(df_train_x)
plt.figure(figsize= (20, 10))
sns.boxplot(data= df_out)
sns.catplot(data= df_out)
plt.xticks(rotation= 90)
plt.show()

### How well does the most important feature distinguish the target?
##### The most important feature is taken as the one most correlated with the target.

In [ ]:
print(f"GINI score of the best feature and the target in the whole dataset: {abs((2 * roc_auc_score(df['Target'], df['K']) - 1)):.4f}")

### How do different models perform using all features?
##### Our baseline is 0.4600 GINI score.

In [ ]:
def print_my_models(X_train, y_train, X_test, y_test, X_train_scaled, X_test_scaled,
                    rf= True, xg= True, lr= True, mlp= True, random_grid= True, scoring= 'roc_auc', cv= 5):
    # Random Forest
    print("\n\033[1m\033[4mRandom Forest results: \033[0m\n")
    rf_model = RandomForestClassifier(random_state= 42)
    rf_param_grid = {'n_estimators': [50, 100, 200, 500],
                     'max_depth': [None, 5, 10, 15],
                     'min_samples_split': [2, 5, 8, 10],
                     'min_samples_leaf': [1, 2, 4]}
    train_test_model(X_train, y_train, X_test, y_test, rf_model, 
                     param_grid= rf_param_grid, random_grid= random_grid, scoring= scoring, cv= cv) if rf else None

    # XGBoosting
    print("\n\033[1m\033[4mXGBoosting results: \033[0m\n")
    xgb_model = xgb.XGBClassifier(random_state= 42)
    xgb_param_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2], 
                      'max_depth': [3, 5, 7, 10], 
                      'min_child_weight': [1, 3, 5, 7], 
                      'gamma': [0.0, 0.1, 0.2, 0.3], 
                      'colsample_bytree': [0.3, 0.5, 0.8, 1.0], 
                      'reg_alpha': [0, 0.1, 1], 
                      'reg_lambda': [1, 1.5, 2]}
    train_test_model(X_train, y_train, X_test, y_test, xgb_model, 
                     param_grid= xgb_param_grid, random_grid= random_grid, scoring= scoring, cv= cv) if xg else None
    
    # Logistic Regression
    print("\n\033[1m\033[4mLogistic Regression results: \033[0m\n")
    lr_model = LogisticRegression(random_state= 42)
    lr_param_grid = {'C': np.logspace(-4, 4, 20),
                     'solver': ['lbfgs', 'newton-cholesky', 'newton-cg', 'sag', 'saga'],
                     'max_iter': [750, 1000, 1500, 2000]}
    train_test_model(X_train_scaled, y_train, X_test_scaled, y_test, lr_model,
                     param_grid= lr_param_grid, random_grid= random_grid, scoring= scoring, cv= cv) if lr else None
    
    # Multi-Layer Perceptron
    print("\n\033[1m\033[4mMulti-Layer Perceptron results: \033[0m\n")
    mlp_model = MLPClassifier(random_state= 42)
    mlp_param_grid = {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 25), (100, 50), (100, 100)],
                      'activation': ['tanh', 'relu', 'logistic'],
                      'solver': ['adam', 'sgd'],
                      'alpha': uniform(0.0001, 0.01),
                      'learning_rate': ['constant', 'adaptive']}
    train_test_model(X_train_scaled, y_train, X_test_scaled, y_test, mlp_model,
                     param_grid= mlp_param_grid, random_grid= random_grid, scoring= scoring, cv= cv) if mlp else None
    
def train_test_model(X_train, y_train, X_test, y_test, model, param_grid, random_grid, scoring, cv):
    if random_grid:
        grid = RandomizedSearchCV(estimator= model, param_distributions= param_grid, scoring= scoring, n_iter= 20, n_jobs= 4, cv= cv, verbose= 4)
    else:
        grid = GridSearchCV(estimator= model, param_grid= param_grid, scoring= scoring, n_jobs= 4, cv= cv, verbose= 4)
    
    grid.fit(X_train, y_train)
    best_params = grid.best_params_
    best_model = grid.best_estimator_
    cv_results = grid.cv_results_
    print(f"Best hyperparameters: {best_params}")
    print(f"Best estimator: {best_model}")
    print(f"Model Cross-Validation AUC: {cv_results['mean_test_score'][grid.best_index_]:.4f} with S.D. {cv_results['std_test_score'][grid.best_index_]}\n")
    
    y_pred_tr = best_model.predict(X_train)
    y_prob_tr = best_model.predict_proba(X_train)[:, 1]
    print(f"Training Set Accuracy: {accuracy_score(y_train, y_pred_tr):.4f}\nTraining Set AUC: {roc_auc_score(y_train, y_prob_tr):.4f}")
    print(f"\n\033[1mTraining Set GINI: {(2 * roc_auc_score(y_train, y_prob_tr) - 1):.4f}\033[0m\n")
    
    y_pred_ts = best_model.predict(X_test)
    y_prob_ts = best_model.predict_proba(X_test)[:, 1]
    print(f"Validation Set Accuracy: {accuracy_score(y_test, y_pred_ts):.4f}\nValidation Set AUC: {roc_auc_score(y_test, y_prob_ts):.4f}")
    print(f"\n\033[1mValidation Set GINI: {(2 * roc_auc_score(y_test, y_prob_ts) - 1):.4f}\033[0m\n")
    
    print(f"Prediction Report:\n {classification_report(y_test, y_pred_ts)}")
    print(f"Confusion Matrix:\n {confusion_matrix(y_test, y_pred_ts)}\n")

##### Trying with all features

In [ ]:
allf_pipeline = Pipeline([
    ("Binary Encoder", OneHotEncoding(["A"], binary= True)),
    ("One-Hot Encoder", OneHotEncoding(["B", "C", "D"])),
    ("Target-Mean Encoder", TargetMeanEncoding(["E"], y_train))
])
scaling_pipeline = Pipeline([
        ("Amount Scalers", CustomScaler(["G", "H", "I"], StandardScaler)),
        ("Count Scalers", CustomScaler(["J", "K", "L", "M", "N", "O", "U", "P"], MinMaxScaler))
])

X_train = allf_pipeline.fit_transform(df_train_x)
X_val = allf_pipeline.transform(df_val_x)
X_train_scaled = scaling_pipeline.fit_transform(X_train)
X_val_scaled = scaling_pipeline.transform(X_val)

print_my_models(X_train, y_train, X_val, y_val, X_train_scaled, X_val_scaled)

##### Does performance increase when we drop uncorrelated features?

In [ ]:
# Threshold %1 or less
dropper_pipeline_t1 = Pipeline([
    ("Feature Dropper", FeatureDropper(["B_1", "D_1", "D_2", "D_3"]))
])

X_train_t1 = dropper_pipeline_t1.fit_transform(X_train)
X_val_t1 = dropper_pipeline_t1.transform(X_val)
X_train_scaled_t1 = dropper_pipeline_t1.fit_transform(X_train_scaled)
X_val_scaled_t1 = dropper_pipeline_t1.transform(X_val_scaled)

print_my_models(X_train_t1, y_train, X_val_t1, y_val, X_train_scaled_t1, X_val_scaled_t1)

In [ ]:
# Threshold %5 or less
dropper_pipeline_t5 = Pipeline([
    ("Feature Dropper", FeatureDropper(["C_1", "L", "C_2", "D_6",
                                        "B_1", "D_1", "D_2", "D_3",
                                        "D_4", "B_2", "D_5", "B_3"]))
])

X_train_t5 = dropper_pipeline_t5.fit_transform(X_train)
X_val_t5 = dropper_pipeline_t5.transform(X_val)
X_train_scaled_t5 = dropper_pipeline_t5.fit_transform(X_train_scaled)
X_val_scaled_t5 = dropper_pipeline_t5.transform(X_val_scaled)

print_my_models(X_train_t5, y_train, X_val_t5, y_val, X_train_scaled_t5, X_val_scaled_t5)

##### Feature Engineering

In [ ]:
def calculate_corr(df, y, gini= False):
    scores = {}
    for feature in df.columns:
        if gini:
            score = 2 * roc_auc_score(y, df[feature]) - 1
        else:
            score = df[feature].corr(y)
        scores[feature] = score
    sorted_values = sorted(scores.items(), key= lambda item: item[1], reverse= True)
    print("Scores:")
    for feature, score in sorted_values:
        print(f"{feature}:\t{score}")
calculate_corr(X_train, y_train)

In [ ]:
logtr_pipeline = Pipeline([
    ("Log", LogTransformFeature(["G", "H", "I","J", "K", "L", "M", "N", "O", "U", "P"])),
    ("Dropper", FeatureDropper(["G", "H", "I", "J", "K", "L", "M", "N", "O", "U", "P"]))
])
X_train_logtr = logtr_pipeline.fit_transform(X_train)
X_val_logtr = logtr_pipeline.transform(X_val)
calculate_corr(X_train_logtr, y_train)

In [ ]:
print_my_models(X_train_logtr, y_train, X_val_logtr, y_val, X_train_logtr, X_val_logtr)

In [ ]:
denc_pipeline = Pipeline([
    ("Binary Encoder", OneHotEncoding(["A"], binary= True)),
    ("One-Hot Encoder", OneHotEncoding(["B", "C"])),
    ("Target-Mean Encoder", TargetMeanEncoding(["D", "E"], y_train))
])
X_train_denc = denc_pipeline.fit_transform(df_train_x)
X_val_denc = denc_pipeline.transform(df_val_x)
X_train_denc_scaled = scaling_pipeline.fit_transform(X_train_denc)
X_val_denc_scaled = scaling_pipeline.transform(X_val_denc)
print_my_models(X_train_denc, y_train, X_val_denc, y_val, X_train_denc_scaled, X_val_denc_scaled)

In [ ]:
intr_pipeline = Pipeline([
    ("Interact", InteractionFeature([ "K", "L", "M", "G"])),
    ("Scaler", CustomScaler(["K_x_L", "K_x_M", "K_x_G", "L_x_M", "L_x_G", "M_x_G"], RobustScaler))
])
X_train_intr = intr_pipeline.fit_transform(X_train_denc)
X_val_intr = intr_pipeline.transform(X_val_denc)
X_train_intr_scaled = scaling_pipeline.fit_transform(X_train_intr)
X_val_intr_scaled = scaling_pipeline.transform(X_val_intr)
print_my_models(X_train_intr, y_train, X_val_intr, y_val, X_train_intr_scaled, X_val_intr_scaled)

In [ ]:
logintr_pipeline = Pipeline([
    ("Log", LogTransformFeature(["G", "H", "I", "J", "K", "L", "M", "N", "O", "U", "P"])),
    ("Dropper", FeatureDropper(["G", "H", "I", "J", "K", "L", "M", "N", "O", "U", "P"])),
    ("Interact", InteractionFeature(["K_log", "L_log", "M_log", "G_log"])),
    ("Scaler", CustomScaler(["K_log_x_L_log", "K_log_x_M_log", "K_log_x_G_log", "L_log_x_M_log", "L_log_x_G_log", "M_log_x_G_log"], MinMaxScaler))])

X_train_logintr = logintr_pipeline.fit_transform(X_train_denc)
X_val_logintr = logintr_pipeline.transform(X_val_denc)
print_my_models(X_train_logintr, y_train, X_val_logintr, y_val, X_train_logintr, X_val_logintr)

##### Trying out test data with the best models. (Highest GINI score)

In [ ]:
best_rf_model = RandomForestClassifier(max_depth=10, min_samples_leaf=4, n_estimators=200, random_state=42)
best_xgb_model = xgb.XGBClassifier(colsample_bytree=1.0, gamma=0.0, learning_rate=0.1, max_depth=3, min_child_weight=1,  random_state=42, reg_alpha= 1, reg_lambda= 2)
best_lr_model = LogisticRegression(C=1438.44988828766, max_iter=2000, random_state=42, solver='sag')
best_mlp_model = MLPClassifier(activation='logistic', alpha=0.0008645723603459421, hidden_layer_sizes=(100, 25), learning_rate='adaptive', random_state=42)

def best_model_results(model, X_train, X_val):
    model.fit(X_train, y_train)
    y_pred_tr = model.predict(X_train)
    y_prob_tr = model.predict_proba(X_train)[:, 1]
    
    start_time = time.time()
    y_pred_ts = model.predict(X_val)
    end_time = time.time()
    y_prob_ts = model.predict_proba(X_val)[:, 1]

    print(f"Training Set Accuracy: {accuracy_score(y_train, y_pred_tr):.4f}")
    print(f"\n\033[1mTraining Set GINI: {(2 * roc_auc_score(y_train, y_prob_tr) - 1):.4f}\033[0m\n")

    print(f"Validation Set Accuracy: {accuracy_score(y_val, y_pred_ts):.4f}")
    print(f"\n\033[1mValidation Set GINI: {(2 * roc_auc_score(y_val, y_prob_ts) - 1):.4f}\033[0m\n")
    
    print(f"Time taken for prediction: {(end_time - start_time):.4f} seconds.")

In [ ]:
print("\n\033[1m\033[4mRandom Forest results: \033[0m\n")
best_model_results(best_rf_model, X_train, X_val)

print("\n\033[1m\033[4mXGBoosting results: \033[0m\n")
best_model_results(best_xgb_model, X_train_denc, X_val_denc)

print("\n\033[1m\033[4mLogistic Regression results: \033[0m\n")
best_model_results(best_lr_model, X_train_logintr, X_val_logintr)

print("\n\033[1m\033[4mMulti-Layer Perceptron results: \033[0m\n")
best_model_results(best_mlp_model, X_train_logtr, X_val_logtr)

In [ ]:
# Choose best Multi-Layer Perceptron model
X_test_allf = allf_pipeline.transform(df_test)
X_test = logtr_pipeline.transform(X_test_allf)

y_pred = best_mlp_model.predict(X_test)
y_prob = best_mlp_model.predict_proba(X_test)[:, 1]
y_pred = pd.DataFrame(y_pred, columns= ['Predictions'])
y_prob = pd.DataFrame(y_prob, columns= ['Probabilities'])

file_name = ""
df_final = pd.read_excel(file_name)
df_final = pd.concat([df_final, y_pred, y_prob], axis= 1)
df_final.to_excel(file_name, index= False)
print("Predictions and Probabilities saved to Test.xlsx.")